In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 라이브러리 호출
import ast
import numpy as np
import pandas as pd

In [ ]:
Input = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Alcohol_Data/INPUT.csv')
Output = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Alcohol_Data/OUTPUT.csv')
User = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Alcohol_Data/USER_SOJU.csv')
#User = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Alcohol_Data/USER_NONSOJU.csv')

0. USER 테이블에서 유저가 선택한 술 데이터 추출하기
- user_info: 설문 유저의 데이터
- golm_list: 유저가 선택한 술 이름 (리스트 형식)
- user_input: 유저가 선택한 술 데이터

In [ ]:
# 설문 유저 데이터 가져오기
user_info = User[(User['unique_id'] == 1) & (User['golajum'].isnull())]

# 설문 유저가 선택한 술 목록을 리스트 형식으로 변경하기
golm = user_info['golm'].apply(lambda x: ast.literal_eval("[" + ", ".join("'" + item.strip() + "'" for item in x.split(',')) + "]") if isinstance(x, str) else x)
golm_list = golm[0]

# 유저가 선택한 술의 데이터 가져오기
user_input = Input[Input['상품명'].isin(golm_list)]

1. 입력받은 술의 좌표 생성
- user_alcohol: 입력받은 술들을 대표하는 좌표값 리스트

In [ ]:
# CASE1) 소주일 경우 - 단/바/향/상
if user_input['주종'].str.lower().eq('소주').any():
  user_alcohol = user_input.loc[user_input['주종'].str.lower() == '소주', ['단맛', '바디감', '향의 강도', '상대 도수']].mean().round(1).tolist()

# CASE2) 맥주/와인일 경우 - 단/신/청/바/상
else:
  user_alcohol = user_input[['단맛', '신맛', '청량감', '바디감', '상대 도수']].mean().round(1).tolist()

2. 가성비 필터링 알고리즘
- Output_Filtering: 필터링에 사용한 Output 데이터셋

In [ ]:
# CASE1) 가성비가 좋은 설문을 선택한 경우:1
if user_info['CE_good_bool'].iloc[0] == 1:
  Output_Filtering = Output[Output['맥주 가성비'] == 1]

# CASE2) 가성비에 상관없는 설문을 선택한 경우:0
else:
  Output_Filtering = Output

3. 도수 설정 (절대 도수 필터링 & 상대 도수 좌표편입)
- avg_alcohol: 사용자가 선택한 술들의 절대 도수의 평균

In [ ]:
# 절대 도수일 경우:0
if user_info['alc_range_bool'].iloc[0] == 0:

  # 절대 도수 평군값 계산
  avg_alcohol = user_input['절대 도수'].mean()

  # 필터링 할 범위 설정
  lower = avg_alcohol - 3
  upper = avg_alcohol + 3

  # 범위에 해당하는 값을 제외하고 나머지 행 삭제
  Output_Filtering = Output_Filtering[(Output_Filtering['절대 도수'] >= lower) & (Output_Filtering['절대 도수'] <= upper)]

  # user_alcohol(입력받은 술들의 좌표값) 에서 상대도수 좌표 삭제하기
  user_alcohol.pop()

# 상대 도수일 경우:1
else:
  pass

4. 유클리드 거리계산

In [ ]:
# Case1-1) 소주이고 절대도수일 경우
if user_input['주종'].str.lower().eq('소주').any() and user_info['alc_range_bool'].iloc[0] == 0:
    selected_columns = ['단맛', '바디감', '향의 강도']

# Case1-2) 소주이고 상대도수일 경우
elif user_input['주종'].str.lower().eq('소주').any() and user_info['alc_range_bool'].iloc[0] == 1:
    selected_columns = ['단맛', '바디감', '향의 강도', '상대 도수']

# Case2-1) 맥주/와인이고 절대도수일 경우
elif user_input['주종'].str.lower().isin(['맥주', '와인']).any() and user_info['alc_range_bool'].iloc[0] == 0:
    selected_columns = ['단맛', '신맛', '청량감', '바디감']

# Case2-2) 맥주/와인이고 상대도수일 경우
elif user_input['주종'].str.lower().isin(['맥주', '와인']).any() and user_info['alc_range_bool'].iloc[0] == 1:
    selected_columns = ['단맛', '신맛', '청량감', '바디감', '상대 도수']

# 선택된 컬럼이 있을 경우 유클리디언 거리 계산 및 추가
if selected_columns:
    # distances = Output_Filtering[selected_columns].apply(lambda row: np.linalg.norm(np.array(row) - np.array(user_alcohol).reshape(1, -1)), axis=1)
    distances = np.linalg.norm(Output_Filtering[selected_columns].values - user_alcohol, axis=1)
    Output_Filtering['거리'] = distances

5. 중요요소(factor) 알고리즘

In [ ]:
# CASE1) 향의 종류를 중요하게 생각하는 경우
if user_info['factor'].iloc[0] == '향의 종류':

    # 교집합의 개수를 계산하는 함수
    def intersection_count(row):
        scent = row['향의 종류']
        flavors = user_info['scent'].iloc[0]
        return sum(1 for flavor in flavors if flavor in scent)

    #  Output_Filtering에서 사용자 선호향(scent)과 술의 '향의 종류'의 일치 정도 계산
    total_flavor_count = Output_Filtering['향의 종류'].apply(lambda flavors: len(flavors))

    # 사용자 선호향(scent)과 술의 '향의 종류' 간의 교집합 개수를 계산하여 비율 계산
    Output_Filtering['가중된 거리'] = Output_Filtering.apply(lambda row: row['거리'] * max(0.2, (1 - intersection_count(row) / total_flavor_count.iloc[0])), axis=1)


# CASE2) 향의 종류 이외를 중요하게 생각하는 경우
else:
    def calculate_weighted_distance(row):
        column_value = row[user_info['factor'].iloc[0]]
        weight = (6 - column_value) / 5  # 5부터 1까지의 가중치를 적용
        return row['거리'] * weight

    # '가중된 거리'를 조절
    Output_Filtering['가중된 거리'] = Output_Filtering.apply(calculate_weighted_distance, axis=1)

6. 최적의 술 추천
- golajum_info: 최종 추천술 정보
- golajum: 최종술 상품명

In [ ]:
# 가중된 거리가 가장 짧은 열 추출
golajum_info = Output_Filtering.loc[Output_Filtering['가중된 거리'].idxmin()]

# 해당 열의 상품명 추출
golajum_name = golajum_info['상품명']
golajum_ab = golajum_info['절대 도수']
golajum_ml = golajum_info['용량']
golajum_price = golajum_info['가격']
golajum_pic = golajum_info['사진']

# USER 테이블에 추천술에 해당 상품명 입력
user_info['golajum'] = golajum_name

# 최종 추천 술 출력
print("당신의 추천술 이름:", golajum_name)
print("당신의 추천술 절대도수:", golajum_ab)
print("당신의 추천술 용량:", golajum_ml)
print("당신의 추천술 가격:", golajum_price)
print("당신의 추천술 사진:", golajum_pic)

당신의 추천술 이름: 금산인삼주
당신의 추천술 절대도수: 12.5
당신의 추천술 용량: 375
당신의 추천술 가격: 4000
당신의 추천술 사진: nan
